In [13]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import fasttext
import re

In [2]:
fake = pd.read_csv('fake.csv')
fake.shape

(1339, 2)

In [25]:
non_fake = pd.read_csv('non-fake.csv')
# non_fake = non_fake.sample(n=2000,random_state=10)

In [26]:
dataset = pd.concat([fake,non_fake])
dataset.sample(10)

,Review,Label
3843,#পুরান_ঢাকা_সমাচার নাম করা কিছু জায়গায় খাইতে গ...,1
6771,!!৩ টকের ফুচকা!!.মুগদা মেডিকেলের পিছের দিকে বল...,1
2876,#ঢাকার_সেরা_সেট_ম্যানু_২০০_টাকায় Laa Eatery Re...,1
1640,কালকে 2 Point 0 Rooftop Caff এর ৫০% অফ অফার প...,1
4172,ফ্রেন্ডদের সাথে খেতে গিয়েছিলাম। অনেক ভালো লাগ...,1
4536,"#BurgerLoverAfter A Long Time, I'm Back With M...",1
4785,বেশ কিছুদিন আগে একটা কন্টেস্ট হয়েছিল। সেই কন্ট...,1
7440,#BurgerLover#BurgerMonth#Uttaraফেব্রুয়ারীকে মা...,1
4459,#বাফেটে_পিৎজা_খেলাম #New_buffetalert The...,1
1929,Buffet Review @Meritage DhakaDecoration যেমন অ...,1


In [27]:
dataset.to_csv('bangla_food_review.csv',index=False)

In [28]:
data = pd.read_csv('bangla_food_review.csv')
data.sample(5)

,Review,Label
1849,"""আপনি কি বনশ্রী-রামপুরা-আফতাবনগর বা এর আশেপাশে...",1
659,#pakkipagol লিখে আপনার অনুভূতি জানিয়ে জিতে নি...,0
5560,বনশ্রীর আফতাবনগরের ভূ-স্বর্গ । একদম নতুন গর্জি...,1
4039,ঝালখোর সেই লেভেলের?বনানীর বাসিন্দা? ভেলপুরি ভা...,1
3387,খিলগাঁওয়ে মুক্তা বিরিয়ানি ঘরের ঝাল ঝাল গরুর চা...,1


In [29]:
data.dropna(inplace=True)
data.shape

(9049, 2)

In [30]:
data.Label.replace(1,"Not_Fake",inplace=True)
data.Label.replace(0,"Fake",inplace=True)
data.sample(5)

,Review,Label
4630,ধানমন্ডির সেরা খিচুড়ি Star Kabab ও ফেল‌ আর দাম...,Not_Fake
7921,#১২২ টাকায় আস্ত মুরগির এত বড় রোস্ট পোলাও জনপ্...,Not_Fake
4415,The Dining Lounge Wari Rooftop onader chowmein...,Not_Fake
6534,মিরপুর এর হাজার হাজার রেস্টুরেন্ট এর মাঝে আজকে...,Not_Fake
7094,রেস্তোরাঁ : butter the bakery and waffle house...,Not_Fake


In [31]:
data["Label"] = "__label__" + data["Label"].astype(str)
data.sample(5)

,Review,Label
6242,Fried Rice with Chicken Tandoori - only 150/=ব...,__label__Not_Fake
6777,"Mexican BBQ Rice BowlPlace: Mexican Spicy, শ্য...",__label__Not_Fake
1743,"আজকে উত্তরা গ্রামের এক বিখ্যাত রেস্টুরেন্টে ""G...",__label__Not_Fake
8330,#আনলিমিটেড_ভাত_ডাল_ভর্তা==১২০টাকা #৮আইটেম_এর_ভ...,__label__Not_Fake
5531,বনশ্রী এবং আফতাবনগরবাসীর জন্য গেট-টুগেদার প্লে...,__label__Not_Fake


In [32]:
data["category_description"] = data["Label"] +" "+ data["Review"]
data.sample(5)

,Review,Label,category_description
5241,East West University এর কাছেই খেতে গেলাম মজাদা...,__label__Not_Fake,__label__Not_Fake East West University এর কাছে...
934,"#Owner post\n#সম্মানিত ফুডিস\n""Gray olive cafe...",__label__Fake,"__label__Fake #Owner post\n#সম্মানিত ফুডিস\n""G..."
702,পৃথিবীর সব সুখ এক দিকে \nআর দুপুর বেলা Burgur ...,__label__Fake,__label__Fake পৃথিবীর সব সুখ এক দিকে \nআর দুপু...
3991,জীবনের প্রথম বার পরিবারকে নিয়ে এতো কম টাকায় উন...,__label__Not_Fake,__label__Not_Fake জীবনের প্রথম বার পরিবারকে নি...
4169,ঢাকা শহরের সব নামকরা খাবারের লিস্ট; দেখেন কোনট...,__label__Not_Fake,__label__Not_Fake ঢাকা শহরের সব নামকরা খাবারের...


In [33]:
def preprocess(sentence):
    sentence = re.sub("['।,]+",' ',sentence)
    sentence = re.sub(r'\n', '', sentence)
    sentence = re.sub(r'[ ]+', ' ', sentence)
    return sentence.strip().lower()

In [34]:
data.category_description = data.category_description.map(preprocess)
data.sample(5)

,Review,Label,category_description
2300,ঈদ আর সাথে প্রচ্চুর বৃষ্টি!অলস অবসর কাটাতে চলল...,__label__Not_Fake,__label__not_fake ঈদ আর সাথে প্রচ্চুর বৃষ্টি!অ...
8,"""অনেক বছর ধরে বড়ই এর আচার খাই নাই.....যদিও এখন...",__label__Fake,"__label__fake ""অনেক বছর ধরে বড়ই এর আচার খাই না..."
6344,মনের শান্তি বড় শান্তি আর সেই শান্তি যদি হয় ভর...,__label__Not_Fake,__label__not_fake মনের শান্তি বড় শান্তি আর সে...
108,Location : Crazy Butler (Satroja) 😍\nIt...,__label__Fake,__label__fake location : crazy butler (satroja...
4843,#নেহারির_ভালোবাসাতে_আমি মিয়া ভাই Miya ...,__label__Not_Fake,__label__not_fake #নেহারির_ভালোবাসাতে_আমি মিয়া...


In [35]:
train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

In [36]:
train_set.category_description

8614    __label__not_fake #ধামাকা_অফার_৪#দ্যা_মুন_লাইট...
5137    __label__not_fake পিৎজ্জা লাভারদের জন্য সুখবরই...
9024    __label__not_fake ৩০০ টাকারসেট মেনু দুই জনেরটে...
5072    __label__not_fake pizza মাস্তান https://www.fa...
2807    __label__not_fake কলেজ থেকে আসার সময় pizzabur...
                              ...                        
5734    __label__not_fake বাই ১ গেট ১ বার্গার যখন মন খ...
5191    __label__not_fake পুরান ঢাকার ঐতিহ্যবাহী খাবার...
5390    __label__not_fake #foodreview#street_foodplace...
860     __label__fake #brownie ❤ this fudgy brownie is...
7270    __label__not_fake number one must try item che...
Name: category_description, Length: 7239, dtype: object

In [37]:
train_set.to_csv('food_review.train',columns=["category_description"],index=False,header=False)
test_set.to_csv('food_review.test',columns=["category_description"],index=False,header=False)

In [38]:
model = fasttext.train_supervised(input='food_review.train')


In [39]:
model.test("food_review.test")

(1610, 0.8875776397515528, 0.8875776397515528)

In [46]:
model.predict('''○বান্ধুবিরা মিলে অনেক দিন পর বের হয়েছিলাম রিক্সায় ঘুরতে কিন্তুু ওরা খালি চাস্ন থাকে treat জন্য আমার আবার সুন্দর সুন্দর sun glass পরতে অনেক ভাল লাগে নতুন একটা চশমা পরে আজকে বের হয়েছিলাম বান্ধুবিরাও বললো েতাকে মানাইসে treat বাহানা আর খুইজা না পাওয়ায় বলতাসে নতুন glass কিনসস সেই খুশিতে treat দে 😒 কি আর করার অই দিন ফুড ব্যাংকে burger fiesta review দেক্লাম খুব reasonable price তাদের ফুডের আর খাবারও মজা আগেও তাদের ফুড আমি test করেছি 😋then burger fiesta গেলাম আমরা অডার করেছিলাম naga wings 🍗naga wings টা খুবই ঝাল ঝাল আমার খুব মজা লাগে জারা ঝাল খেতে পারেন must be try করবেন and সাথে অবশ্যই cold coffee নিতে হবে তাহলে test টা আরও মজা লাগে 😋 i love this burger fiesta s french fry 🍟🧡তাদের french fry টা খুবই crispy হয় and last one chicken sandwich as a good test 🥪 1.naga wings 4 pics/🍗99 tk. 2.chicken sandwich 4 pics🥪/99 tk. 3.french fry /100 tk 🍟 4.cold coffee 70 tk🍹''')

(('__label__not_fake',), array([0.73467517]))

In [41]:
model.save_model('bangla_food_review.model')